In [2]:
import pandas as pd

In [3]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [4]:
costed_steps = structure[structure.step_cost > 0]

In [5]:
def f(x):
    passed = x[x.action == "passed"]
#     print(passed)
    two_weeks = 10*3600*24
    border = passed.time.max() - two_weeks
    
    return len([1 for t in passed.time if t >= border])

In [7]:
from tqdm import tqdm
tqdm.pandas()

X = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)

100%|██████████| 9426/9426 [00:12<00:00, 770.57it/s]


In [8]:
Y = targets[targets.user_id.isin(X.index)]

In [13]:
X = X.values
Y = Y.passed.values

In [18]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier()

In [20]:
classifier.fit(X.reshape(len(X), 1), Y.reshape(len(Y), 1))

/Library/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [22]:
from sklearn.metrics import accuracy_score

In [25]:
def prepare(df):
    X = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
    X = X.values.reshape(len(X), 1)
    return X

In [28]:
X_test = prepare(pd.read_csv("kaggle/user_activity_test.csv"))
predicted = classifier.predict(X_test)

100%|██████████| 2413/2413 [00:02<00:00, 992.29it/s] 
